# Analyze Financial Data with LLMs

Get AI-powered insights from financial reports and data.


In [7]:
%pip install -qU pixeltable openai yfinance


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os, getpass
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')


In [ ]:
import pixeltable as pxt
from pixeltable.functions import openai
import yfinance as yf

In [9]:
# Step 1: Define UDF to fetch stock data
@pxt.udf
def get_stock_data(ticker: str) -> str:
    """Fetch financial summary for a stock ticker."""
    stock = yf.Ticker(ticker)
    info = stock.info
    return f"""Company: {info.get('shortName')}
Current Price: ${info.get('currentPrice', 'N/A')}
Market Cap: ${info.get('marketCap', 0) / 1e9:.2f}B
P/E Ratio: {info.get('trailingPE', 'N/A')}
52-Week High: ${info.get('fiftyTwoWeekHigh', 'N/A')}
52-Week Low: ${info.get('fiftyTwoWeekLow', 'N/A')}"""


In [10]:
# Step 2: Create analysis table
pxt.create_dir('finance', if_exists='ignore')
stocks = pxt.create_table('finance.analysis', {'ticker': pxt.String, 'query': pxt.String}, if_exists='ignore')


In [11]:
# Step 3: Auto-fetch data and analyze with LLM
stocks.add_computed_column(data=get_stock_data(stocks.ticker), if_exists='ignore')
stocks.add_computed_column(if_exists='ignore',
    analysis=openai.chat_completions(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': 
            f"Data:\n{stocks.data}\n\nQuestion: {stocks.query}"}]
    ).choices[0].message.content
)


No rows affected.

In [ ]:
# Step 4: Analyze a stock
stocks.insert([{'ticker': 'NVDA', 'query': 'Is this stock overvalued?'}])

ExprEvalError: Expression evaluation failed with an error of type `pixeltable.exceptions.Error`:
chat_completions(model='gpt-4o-mini', messages=[{'role': 'user', 'content': 'Data:\ndata\n\nQuestion: query'}])

In [ ]:
# View analysis results
stocks.select(stocks.ticker, stocks.analysis).head()


**What's Happening:**
- UDF fetches live financial data via yfinance
- Data stored as computed column
- LLM analyzes data and answers questions
- Each insert triggers full pipeline automatically

**Variation:** Compare multiple stocks:
```python
stocks.insert([
    {'ticker': 'NVDA', 'query': 'Growth potential?'},
    {'ticker': 'AMD', 'query': 'Growth potential?'}
])
```

**Next:** `monitor-brand-sentiment-from-social-media.ipynb` • `compare-product-descriptions-across-models.ipynb`
